In [1]:
import pandas as pd
import numpy as np
import glob
from pandarallel import pandarallel
from bs4 import BeautifulSoup, SoupStrainer
import requests
import os
import random


In [17]:
os.makedirs("data/github_clean/linked_issues", exist_ok=True)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
def grabIssue(repo, issue_number):
    try:
        scrape_url = f"https://github.com/{repo}/issues/{issue_number}"
        product = SoupStrainer('a')
        sesh = requests.Session() 
        page = sesh.get(scrape_url)
        page_text = str(page.text)
        if "Please wait a few minutes before you try again" in page_text:
            print('pausing, rate limit hit')
            time.sleep(120)
            page = sesh.get(scrape_url)
            page_text = str(page.text)
            
        soup = BeautifulSoup(page.content,parse_only = product,features="html.parser")
        pr_links = soup.find_all("a", attrs={"data-hovercard-type":'pull_request'})
        pr_link = pr_links[0]['href']
        return pr_link
    except Exception as e:
        error = str(e)
        return error

In [5]:
files = glob.glob('data/inputs/linked_issues/*')

In [6]:
random.shuffle(files)

In [7]:
i = 0
existing_issues = pd.DataFrame()
for i in range(1000):
    i+=1
    existing_issues = pd.concat([existing_issues, pd.read_parquet(f'data/github_clean/linked_issues2/linked_issue_{i}.parquet')])

existing_issues.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
for file in files:
    check_issues = pd.read_parquet(file)
    check_issues = pd.merge(check_issues.drop('linked_pr', axis = 1), existing_issues, how = 'left')
    fname = file.split("/")[-1]
    print(fname)
    if fname in os.listdir('data/github_clean/linked_issues'):
        continue
    else:
        while check_issues.dropna().shape[0] != check_issues.shape[0]:
            na_df = check_issues[check_issues['linked_pr'].isna()]
            print(f"{na_df.shape[0]} remaining in {file}")
            # update file
            apply_inds = na_df.sample(min(10, na_df.shape[0])).index
            check_issues.loc[apply_inds, 'linked_pr'] = check_issues.loc[apply_inds].apply(
                lambda x: grabIssue(x['repo_name'], x['potential_issues']), axis = 1)
            ## update with new changes
            check_issues.to_parquet(f'data/github_clean/linked_issues/{fname}')

In [ ]:
linked_issues = pd.concat([pd.read_parquet(file, index_col = 0) for file in glob.glob('data/github_clean/linked_issues/*')])

In [ ]:
linked_issues = linked_issues[linked_issues['linked_pr'] != 'list index out of range']

In [ ]:
linked_issues['pr_number'] = linked_issues['linked_pr'].apply(lambda x: x.split("/")[-1])

In [ ]:
linked_issues.reset_index(drop = True, inplace = True)

In [ ]:
correct = linked_issues[linked_issues['pr_number'].apply(lambda x: '#' in x)].index
linked_issues.loc[correct, 'pr_number'] = linked_issues.loc[correct, 'pr_number'].apply(lambda x: x.split("#")[0])

In [ ]:
linked_issues.to_parquet('data/merged_data/linked_issues.parquet')